In [2]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import pandas as pd

In [4]:
#define constants
#unrolled through 28 time steps
time_steps=100
#hidden LSTM units
num_units=128
#rows of 28 pixels
n_input=12
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 10 classes(0-9).
n_classes=6
#size of batch
batch_size=128

In [5]:
train_data = pd.read_csv("train_data.csv").dropna()
test_data = pd.read_csv("test_data.csv")

In [6]:
y_train = train_data["action_id"].apply(lambda x:x[:3])
y_test= test_data["action_id"].apply(lambda x:x[:3])

In [7]:
encoding = {'jog': 0, 'dws': 1, 'ups': 2, 'wlk': 3, 'sit': 4, 'std': 5}
y_train = y_train.apply(lambda x: encoding [x])
y_test = y_test.apply(lambda x: encoding [x])

In [8]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [9]:
temp_input = train_data.drop(["action_id"] , axis = 1)
temp_input = temp_input.drop(["user_id"] , axis = 1)
temp_input = temp_input.drop(["time_50hz"] , axis = 1)
test_input = test_data.drop(["action_id"] , axis = 1)
test_input = test_input.drop(["user_id"] , axis = 1)
test_input = test_input.drop(["time_50hz"] , axis = 1)

In [10]:
temp_X = temp_input[:1240503]
temp_Y = y_train[:1240503]
test_X = test_input[:170000]
test_Y = y_test[:170000]

In [11]:
set(test_Y)

{0, 1, 2, 3, 4, 5}

In [12]:
temp_X.shape , temp_Y.shape , test_X.shape , test_Y.shape

((1240503, 12), (1240503, 6), (170000, 12), (170000, 6))

In [13]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

In [14]:
#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

In [15]:
#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [16]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

In [20]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [21]:
temp_input[0:batch_size]

,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z
0,1.528132,-0.733896,0.696372,0.741895,0.669768,-0.031672,0.316738,0.778180,1.082764,0.294894,-0.184493,0.377542
1,1.527992,-0.716987,0.677762,0.753099,0.657116,-0.032255,0.842032,0.424446,0.643574,0.219405,0.035846,0.114866
2,1.527765,-0.706999,0.670951,0.759611,0.649555,-0.032707,-0.138143,-0.040741,0.343563,0.010714,0.134701,-0.167808
3,1.516768,-0.704678,0.675735,0.760709,0.647788,-0.041140,-0.025005,-1.048717,0.035860,-0.008389,0.136788,0.094958
4,1.493941,-0.703918,0.672994,0.760062,0.647210,-0.058530,0.114253,-0.912890,0.047341,0.199441,0.353996,-0.044299
5,1.476302,-0.700807,0.669443,0.760912,0.644835,-0.072116,0.187742,-0.763656,0.226057,0.168241,0.145906,0.012455
6,1.455153,-0.694408,0.662593,0.763300,0.639930,-0.088666,0.343096,-0.803820,0.278468,0.079382,-0.026344,-0.195590
7,1.441702,-0.690710,0.656459,0.764380,0.637084,-0.099229,0.176202,-0.172756,0.056415,0.069360,0.072678,-0.102920
8,1.443440,-0.691905,0.651196,0.763796,0.638005,-0.097803,0.274786,0.446585,-0.132766,0.072889,0.079921,-0.075323
9,1.443071,-0.693039,0.638198,0.763042,0.638878,-0.097993,0.633672,0.316372,-0.115137,0.098347,-0.017021,-0.197310


In [22]:
batch_x = temp_X[0:batch_size]
batch_x = np.array(batch_x)
batch_x=batch_x.reshape((batch_size,time_steps,n_input))
batch_y = temp_Y[0:batch_size]

In [24]:
temp_X.shape , batch_x.shape , batch_y.shape , temp_X.shape , temp_Y.shape

((1240503, 12), (128, 100, 12), (128, 6), (1240503, 12), (1240503, 6))

In [ ]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=0
    while iter<100:
        batch_x = temp_X[iter*batch_size*time_steps:(iter+1)*batch_size*time_steps]
        batch_y = temp_Y[iter*batch_size:(iter+1)*batch_size]
        batch_x = np.array(batch_x)
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))
#         print(batch_x.shape)

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1


In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/" )


In [ ]:
mnist.validation.images.shape

In [ ]:
vld_data = mnist.validation.images.reshape((-1 , 28 , 28))

In [ ]:
test_X = np.array(test_X)
print(test_X.shape)
test_X=test_X.reshape((3400,time_steps,n_input))

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    test_X=test_X.reshape((3400,time_steps,n_input))
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_X, y: test_Y}))